In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import nengolib
import numpy as np
import pandas as pd
import random
import scipy as sp
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from itertools import islice
from IPython.display import clear_output
import pytry
import pickle

Generate and Save Input Data - Only run once.

In [ ]:
target_dt = 1.0/30

data = pd.read_csv("multidata.csv", low_memory=False)
x = np.array(data.iloc[:,11:195]).astype(float) #array of data for purple child (points in space for each frame)
labs = np.array(data.iloc[:,443]).astype(str) #array of labels (purple child annotations, engagement) 218/443
a = []
for i in range(x.shape[1]):
    y = pd.Series(x[:,i])
    if i == 180:               # add these three lines
        y[y>0]-=np.pi*2   # add these three lines
        y += np.pi            # add these three lines
    z = y.interpolate(limit_direction='both')
    a.append(z)
a = pd.DataFrame(a)
a = a.dropna()
a = np.array(a).T
    
def extract_pattern(start, end, target_dt): #function to extract consecutive frames with the same label 
    pattern = np.array(a[start:end,:]).astype(float)
    frames = np.array(data.iloc[start:end,9]).astype(int)
    timestamp = np.array(data.iloc[start:end,0]).astype(str)

    good_indices = frames != -1
    frames = frames[good_indices]
    pattern = pattern[good_indices]

    fps = 30.0
    t_sample = (frames - frames[0])/fps

    t = np.arange(int(t_sample[-1]/target_dt))*target_dt

    result = []
    for i in range(pattern.shape[1]):       
        p = np.interp(t, t_sample, pattern[:,i])
        result.append(p)
    result = np.array(result).T

    return timestamp, result

start=[]
start.append(0)
end=[]
label = []
for i in range(1, (len(labs)-1)):
    if labs[i]!=labs[i-1]:
        start.append(i)
    if labs[i]!=labs[i+1]:
        end.append(i)
        label.append(labs[i])

t_noplay=[]
p_noplay=[]
t_goal=[]
p_goal=[]
t_aim=[]
p_aim=[]

for i in range(1,(len(start)-1)):
    try:
        if label[i]==('noplay'):
            ts, pi = extract_pattern(start[i], end[i], target_dt=target_dt)
            t_noplay.append(ts)
            p_noplay.append(pi)
        if label[i]==('goaloriented'):
            ts, pi = extract_pattern(start[i], end[i], target_dt=target_dt)
            t_goal.append(ts)
            p_goal.append(pi)
        if label[i]==('aimless'):
            ts, pi = extract_pattern(start[i], end[i], target_dt=target_dt)
            t_aim.append(ts)
            p_aim.append(pi)
    except IndexError:
        print('empty pattern')
        
i=0
goal = []
noplay = []
aim = []
for entry in t_goal:
    goal.append([t_goal[i],p_goal[i]])
    i = i+1
i=0
for entry in t_noplay:
    noplay.append([t_noplay[i],p_noplay[i]])
    i = i+1
i=0
for entry in t_aim:
    aim.append([t_aim[i],p_aim[i]])
    i = i+1
    
np.save('goal.npy', goal)
np.save('noplay.npy', noplay)
np.save('aim.npy', aim)

Set up network.

In [2]:
class PatternInterpolationTrain(pytry.Trial):
    def params(self):
        self.param('number of dimensions', n_dims=2),
        self.param('length of training pattern', len_train=10),
        self.param('number of epochs', n_epoch=1),
        self.param('seed', p_seed=1),
        
    def evaluate(self, param): #function to fill in missing data points by interpolating
        import nengo
        goal = list(np.load('goal.npy'))
        noplay = list(np.load('noplay.npy'))
        aim = list(np.load('aim.npy'))
        
        seed=param.p_seed

        dt = 0.001
        target_dt = 1.0/30
        D = param.n_dims
        classify_score = {}
        accuracy = {}

        p_goal2 = random.sample(goal, len(goal))
        p_noplay2 = random.sample(noplay, len(noplay))
        p_aim2 = random.sample(aim, len(aim))
        p_aim2 = np.array(p_aim2)

        goal_train_ts = np.array(p_goal2[:(int(len(p_goal2)*0.8))]) 
        goal_test_ts = np.array(p_goal2[(int(len(p_goal2)*0.8)):]) 
        goal_train_list=list(goal_train_ts[:,1])
        goal_test_list=list(goal_test_ts[:,1])
        goal_train_time=list(goal_train_ts[:,0])
        goal_test_time=list(goal_test_ts[:,0])

        noplay_train_ts = np.array(p_noplay2[:(int(len(p_noplay2)*0.8))]) 
        noplay_test_ts = np.array(p_noplay2[(int(len(p_noplay2)*0.8)):]) 
        noplay_train_list=list(noplay_train_ts[:,1])
        noplay_test_list=list(noplay_test_ts[:,1])
        noplay_train_time=list(noplay_train_ts[:,0])
        noplay_test_time=list(noplay_test_ts[:,0])
        
        for p in noplay_train_list:
            if p.size > 0:
                pass
            else:
                noplay_train_list.remove(p)

        for p in noplay_test_list:
            if p.size >0:
                pass
            else:
                noplay_test_list.remove(p)

        train_all = np.vstack(goal_train_list+noplay_train_list)
        pca_model = PCA(n_components=D).fit(train_all)

        train_time_all = np.hstack(goal_train_time+noplay_train_time)
        test_time_all = np.hstack(goal_test_time+noplay_test_time)

        goal_train_pca = np.vstack([pca_model.transform(p) for p in goal_train_list])
        noplay_train_pca = np.vstack([pca_model.transform(p) for p in noplay_train_list])

        np.save('%s_goal_train_pca.npy' % seed, goal_train_pca)
        np.save('%s_noplay_train_pca.npy' % seed, noplay_train_pca)
        
########## TESTING PATTERNS - ONE SEGMENT AT A TIME #########
        goal_test_pca2=[]
        noplay_test_pca2=[]
        aim_test_pca2=[]

        for i in range(len(goal_test_list)):
            goal_test_pca2.append(pca_model.transform(goal_test_list[i]))

        for i in range(len(noplay_test_list)):
            noplay_test_pca2.append(pca_model.transform(noplay_test_list[i]))

            
        aim_list=list(p_aim2[:,1])
        aim_time=list(p_aim2[:,0])
        for i in range(len(aim_list)):
            aim_test_pca2.append(pca_model.transform(aim_list[i]))
            
            
        np.save('%s_goal_test_pca2.npy' % seed, goal_test_pca2)
        np.save('%s_noplay_test_pca2.npy' % seed, noplay_test_pca2)
        np.save('%s_aim_test_pca2.npy' % seed, aim_test_pca2)
        np.save('%s_goal_test_time.npy' % seed, goal_test_time)
        np.save('%s_noplay_test_time.npy' % seed, noplay_test_time)
        np.save('%s_aim_time.npy' % seed, aim_time)
#############################################################

        T_train = param.len_train   # number of seconds to train on for each class
        T_test = 100    # number of seconds to test on for each class
        
        goal_train_time2 = [item for sublist in goal_train_time for item in sublist]
        noplay_train_time2 = [item for sublist in noplay_train_time for item in sublist]

        N_frames = int(T_train*30)
        train_timestamps = np.hstack([goal_train_time2[:(N_frames*param.n_epoch)], noplay_train_time2[:(N_frames*param.n_epoch)]])
        
############ TRAINING WITH 80% ############ 
        print('Run Number: ', int(param.p_seed)+1)
        print('Testing with 80% Training Data')
    
        num_epochs = param.n_epoch
        batch_size = int(T_train*30)
        batch = 0
        theta = 15

        for epoch in range(num_epochs):
            training = np.vstack([goal_train_pca[batch:int(batch+batch_size)], noplay_train_pca[batch:int(batch+batch_size)]])
            batch += batch_size

            training_net = nengo.Network(seed=seed)#param.seed)
            with training_net:
                rw = []
                for i in range(D):
                    process = nengo.processes.PresentInput(np.hstack([goal_train_pca[:,i], noplay_train_pca[:,i]]), 
                                                               presentation_time=1.0/30)
                    rw.append(nengolib.networks.RollingWindow(theta=theta, n_neurons=3000, 
                                                              process=process, 
                                                              neuron_type=nengo.Direct()))


                node_pool = nengo.Node(None, size_in=rw[0].state.size_out*D)

                start = 0
                for r in rw:
                    nengo.Connection(r.state, node_pool[start:start+r.state.size_out])
                    start += r.state.size_out



                stim = nengo.Node(nengo.processes.PresentInput(training, presentation_time=1.0/30))
                assert stim.size_out == D
                for i in range(D):
                    nengo.Connection(stim[i], rw[i].input, synapse=None)

                p_node_pool = nengo.Probe(node_pool, sample_every = 0.1)


            sim = nengo.Simulator(training_net)
            with sim:
                sim.run(T_train*2)  


            pool_model = nengo.Network()
            with pool_model:
                pool = nengo.Ensemble(n_neurons=3000, dimensions=node_pool.size_out,
                                      neuron_type=nengo.LIFRate(), seed=seed)
            pool_sim = nengo.Simulator(pool_model)

            import nengo.utils.ensemble

            _, a = nengo.utils.ensemble.tuning_curves(pool, pool_sim, inputs=sim.data[p_node_pool])

            if epoch == 0:
                a_goal = a[:(int(len(a)/2))]
                a_noplay = a[(int(len(a)/2)):]
            else:
                a_goal = np.concatenate((a_goal, a[:(int(len(a)/2))]))
                a_noplay = np.concatenate((a_noplay, a[(int(len(a)/2)):]))

        a_out = np.vstack([a_goal, a_noplay])


        N = int((len(a_out))/2) #int(T_train*1000)
        target = np.hstack([np.ones(N), -np.ones(N)]).reshape(-1, 1)
        dec, info = nengo.solvers.LstsqL2(reg=0.1)(a_out, target)
        
        np.save('%s_decoder.npy' % seed, dec)
        np.save('%s_node_pool.npy' % seed, node_pool.size_out)
        
        v = np.dot(a_out, dec)


        train_timestamps = train_timestamps[0::2]

        output_goal_train = {"output_goal_train":{"time":train_timestamps[:N], "output":v[:N]}}
            
        output_noplay_train = {"output_noplay_train":{"time":train_timestamps[N:], "output":v[N:]}}

############ SAVE DATA ############ 
       
        results = {**output_goal_train, **output_noplay_train}
        #, **output_goal_test, **output_noplay_test, **output_aimless}

        return results            


Run network.

In [3]:
for seed in range(20):
    PatternInterpolationTrain().run(seed=seed, p_seed=seed, len_train=500, n_epoch=3, data_dir='Training_500x3', data_format="npz")

running PatternInterpolationTrain#20190622-121522-7e04e701


C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:51: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


Run Number:  1
Testing with 80% Training Data


C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\signal\filter_design.py:1551: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  "results may be meaningless", BadCoefficients)
C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\nengo\utils\numpy.py:79: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  v = a[inds]
C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\nengolib\signal\system.py:717: UserWarning: Filtering with non-SISO systems is an experimental feature that may not behave as expected.
  "expected.", UserWarning)


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

...

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

seed = 19
n_dims = 2
len_train = 500
n_epoch = 3
p_seed = 19
output_goal_train = {'time': array(['1500027211', '1500027211', '1500027211', ..., '1499425437',
       '1499425437', '1499425437'], dtype='<U10'), 'output': array([[ 0.53869824],
       [ 0.53388949],
       [ 0.53092659],
       ...,
       [ 0.00597984],
       [-0.06129033],
       [-0.07607864]])}
output_noplay_train = {'time': array(['1499425437', '1499425437', '1499425437', ..., '1500030025',
       '1500030025', '1500030025'], dtype='<U10'), 'output': array([[ 0.30581161],
       [ 0.19417895],
       [-0.09268989],
       ...,
       [ 0.33434215],
       [ 0.36927526],
       [ 0.39830303]])}


In [ ]:
#for seed in range(20):
#    PatternInterpolationTrial().run(seed=seed, p_seed=seed, len_train=10, n_epoch=1, data_dir='theta15')
#    clear_output()